In [2]:
from datetime import datetime, timedelta
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import hour, mean
import pyspark
import random
sc = SparkContext.getOrCreate()

In [3]:
spark = pyspark.sql.SparkSession.builder.appName("auctions").getOrCreate()
df_auctions = spark.read.csv('auctions.csv', header=True).limit(10000)
rdd_auctions = df_auctions.rdd
rdd_auctions.take(5)

[Row(date='2019-04-23 18:58:00.842116', device_id='2564673204772915246', ref_type_id='1', source_id='0'),
 Row(date='2019-04-23 18:58:01.530771', device_id='4441121667607578179', ref_type_id='7', source_id='0'),
 Row(date='2019-04-23 18:58:01.767562', device_id='7721769811471055264', ref_type_id='1', source_id='0'),
 Row(date='2019-04-23 18:58:02.363468', device_id='6416039086842158968', ref_type_id='1', source_id='0'),
 Row(date='2019-04-23 18:58:02.397559', device_id='1258642015983312729', ref_type_id='1', source_id='0')]

In [4]:
def didInstall():
    return bool(random.getrandbits(1))

In [5]:
rdd_auctions = rdd_auctions.map(lambda x: (x[1], (datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f"), didInstall())))

In [6]:
rdd_train = rdd_auctions.groupByKey().mapValues(list).mapValues(sorted)

In [7]:
rdd_train.take(20)

[('2564673204772915246',
  [(datetime.datetime(2019, 4, 23, 18, 48, 20, 500673), False),
   (datetime.datetime(2019, 4, 23, 18, 58, 0, 842116), True)]),
 ('4441121667607578179',
  [(datetime.datetime(2019, 4, 23, 18, 58, 1, 530771), True),
   (datetime.datetime(2019, 4, 23, 19, 8, 10, 57648), False),
   (datetime.datetime(2019, 4, 23, 19, 18, 9, 722451), False)]),
 ('7721769811471055264',
  [(datetime.datetime(2019, 4, 23, 18, 58, 1, 767562), True),
   (datetime.datetime(2019, 4, 23, 19, 50, 40, 208816), False)]),
 ('6416039086842158968',
  [(datetime.datetime(2019, 4, 23, 18, 58, 2, 363468), False)]),
 ('1258642015983312729',
  [(datetime.datetime(2019, 4, 23, 18, 58, 2, 397559), False)]),
 ('6707090658317158573',
  [(datetime.datetime(2019, 4, 23, 18, 58, 2, 675804), False),
   (datetime.datetime(2019, 4, 23, 20, 58, 10, 905109), False)]),
 ('8869722088125970841',
  [(datetime.datetime(2019, 4, 23, 18, 58, 2, 848212), False),
   (datetime.datetime(2019, 4, 23, 19, 7, 49, 671153), Fal

In [8]:
def get_installation_timeDeltas(dateList):
    """
    esta funcion recibe una lista de fechas ordenada de las apariciones de un ID
    la idea es devolver una lista de tuplas, en la que cada tupla sea asi:
    valor 1 = fecha en que aparecio en un auction
    valor 2 = distancia temporal hasta la siguiente fecha donde convirtió (0 segundos: si convirtió en esa fecha,
                                                                            3 días (en segundos): si no convirtió 
                                                                            en una fecha posterior)
    Esta tupla solo se inserta en la lista si en la fecha donde se esta parado no hubo una instalación
    """
    distancias = []
    longitud_actual = len(dateList)
    """
    En lastConversionIndexFound guardo el indice de la última conversión encontrada, 
    no quiero iterar n veces si aplica para otras fechas
    """
    lastConversionIndexFound = None
    for x in range(longitud_actual):
        if(dateList[x][1] == True):
            continue
        if(lastConversionIndexFound is not None and dateList[x][1] == False and x <= lastConversionIndexFound):
            distancias.append([dateList[x][0],
                                    (dateList[lastConversionIndexFound][0] - dateList[x][0]).total_seconds()])
            continue
        for y in range(x, longitud_actual):
            if(y == longitud_actual - 1 and dateList[y][1] != True):
                distancias.append([dateList[x][0],
                                            timedelta(days=3).total_seconds()
                                        ])
                break
            elif(dateList[y][1] == True):
                distancias.append([dateList[x][0],
                                    (dateList[y][0] - dateList[x][0]).total_seconds()
                                    ])
                lastConversionIndexFound = y
                break
        
    return distancias

In [9]:
rdd_train_test = rdd_train.flatMapValues(get_installation_timeDeltas)

In [10]:
rdd_train_test.take(30)

[('2564673204772915246',
  [datetime.datetime(2019, 4, 23, 18, 48, 20, 500673), 580.341443]),
 ('4441121667607578179',
  [datetime.datetime(2019, 4, 23, 19, 8, 10, 57648), 259200.0]),
 ('4441121667607578179',
  [datetime.datetime(2019, 4, 23, 19, 18, 9, 722451), 259200.0]),
 ('7721769811471055264',
  [datetime.datetime(2019, 4, 23, 19, 50, 40, 208816), 259200.0]),
 ('6416039086842158968',
  [datetime.datetime(2019, 4, 23, 18, 58, 2, 363468), 259200.0]),
 ('1258642015983312729',
  [datetime.datetime(2019, 4, 23, 18, 58, 2, 397559), 259200.0]),
 ('6707090658317158573',
  [datetime.datetime(2019, 4, 23, 18, 58, 2, 675804), 259200.0]),
 ('6707090658317158573',
  [datetime.datetime(2019, 4, 23, 20, 58, 10, 905109), 259200.0]),
 ('8869722088125970841',
  [datetime.datetime(2019, 4, 23, 18, 58, 2, 848212), 259200.0]),
 ('8869722088125970841',
  [datetime.datetime(2019, 4, 23, 19, 7, 49, 671153), 259200.0]),
 ('6405811806780450397',
  [datetime.datetime(2019, 4, 23, 18, 58, 3, 532112), 259200.

In [11]:
timedelta(days=3).total_seconds()

259200.0

In [12]:
#Auctions date, count, min, max
"""
00:30:00	1546960	NaN	1546960.0
07:00:00	251925	251925.0	NaN
16:00:00	1269111	NaN	1269111.0
17:00:00	1257409	1257409.0	NaN
19:00:00	1291646	NaN	1291646.0
19:30:00	1288588	1288588.0	NaN
20:30:00	1323592	NaN	1323592.0
21:30:00	1294431	1294431.0	NaN
"""

'\n00:30:00\t1546960\tNaN\t1546960.0\n07:00:00\t251925\t251925.0\tNaN\n16:00:00\t1269111\tNaN\t1269111.0\n17:00:00\t1257409\t1257409.0\tNaN\n19:00:00\t1291646\tNaN\t1291646.0\n19:30:00\t1288588\t1288588.0\tNaN\n20:30:00\t1323592\tNaN\t1323592.0\n21:30:00\t1294431\t1294431.0\tNaN\n'

In [31]:
auctionsMax = datetime.strptime("00:30:00", "%H:%M:%S")
auctionsMin = datetime.strptime("07:00:00", "%H:%M:%S")
installsMin = datetime.strptime("07:00:00", "%H:%M:%S")
installsMax = datetime.strptime("23:00:00", "%H:%M:%S")
auctionsMin

datetime.datetime(1900, 1, 1, 7, 0)

In [39]:
date = rdd_auctions.take(5)[0][1][0] 
(auctionsMin - datetime(year = 1900, month = 1, day = 1, hour = date.hour, minute = date.minute)).total_seconds()

-43080.0

In [ ]:
def get_timeDeltas_toMax(dateList, maxTime, minTime):
    """
    esta funcion recibe una lista de fechas ordenada de las apariciones de un ID
    la idea es devolver una lista de tuplas, en la que cada tupla sea asi:
    valor 1 = fecha en que aparecio en un auction
    valor 2 = distancia temporal hasta el siguiente pico de actividad (sea auctions o installs)
    """
    distancias = []
    longitud_actual = len(dateList)
    """
    En lastConversionIndexFound guardo el indice de la última conversión encontrada, 
    no quiero iterar n veces si aplica para otras fechas
    """
    for x in range(longitud_actual):
        date = dateList[x][0]
        datetime(year = 1900, month = 1, day = 1, hour = date.hour, minute = date.minute) - maxTime
        if(dateList[x][1] == True):
            continue
        if(lastConversionIndexFound is not None and dateList[x][1] == False and x <= lastConversionIndexFound):
            distancias.append([dateList[x][0],
                                    (dateList[lastConversionIndexFound][0] - dateList[x][0]).total_seconds()])
            continue
        for y in range(x, longitud_actual):
            if(y == longitud_actual - 1 and dateList[y][1] != True):
                distancias.append([dateList[x][0],
                                            timedelta(days=3).total_seconds()
                                        ])
                break
            elif(dateList[y][1] == True):
                distancias.append([dateList[x][0],
                                    (dateList[y][0] - dateList[x][0]).total_seconds()
                                    ])
                lastConversionIndexFound = y
                break
        
    return distancias